In [23]:
import numpy as np
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import math
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
# from bayes_opt import BayesianOptimization
# from xgboost import XGBRegressor
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
data = pd.read_csv('../datasets/baseball/FG_2010_2018_inning_6_load.csv')

## Explore the Data

In [6]:
data.head(2)

,Season,Name,Team,G,Age,PA,HR,R,RBI,SB,...,f_val_SB_rate,f_val_rate,ny_f_val_rate,HR_rate_std,R_rate_std,RBI_rate_std,SB_rate_std,AVG_std,f_val_std,ny_f_val_std
0,2017,Mookie Betts,Red Sox,153,24,712,24,101,102,26,...,1.83,2.43,0.745460,0.077212,1.205684,1.226771,1.542399,-0.087713,3.964353,12.516023
1,2017,Christian Yelich,Marlins,156,25,695,18,100,81,16,...,0.71,0.77,0.816703,-0.543295,1.037601,0.094554,0.596226,0.534600,1.719685,10.721773


In [7]:
data.describe()

,Season,G,Age,PA,HR,R,RBI,SB,BB%,K%,...,f_val_SB_rate,f_val_rate,ny_f_val_rate,HR_rate_std,R_rate_std,RBI_rate_std,SB_rate_std,AVG_std,f_val_std,ny_f_val_std
count,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,...,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000,1154.000000
mean,2013.550260,141.068458,28.214038,579.262565,18.353553,72.337955,69.570191,10.006932,0.085828,0.183038,...,0.049159,0.195347,0.029685,0.106438,0.175737,0.127342,0.044009,0.127762,0.581287,0.260231
std,2.314325,16.168980,3.662962,86.555025,9.940838,18.773729,22.253659,11.004136,0.030436,0.056472,...,1.147934,1.342655,0.521885,1.015281,0.980659,1.006717,1.014020,0.971037,3.076915,3.240759
min,2010.000000,93.000000,19.000000,400.000000,0.000000,26.000000,17.000000,0.000000,0.021000,0.058000,...,-1.000000,-2.620000,-1.000000,-2.080068,-2.711601,-2.717538,-0.969940,-3.869888,-7.862697,-7.862697
25%,2012.000000,130.000000,26.000000,511.000000,11.000000,58.000000,52.000000,2.000000,0.063000,0.142000,...,-0.770000,-0.727500,-0.366880,-0.695430,-0.534081,-0.627220,-0.666457,-0.535968,-1.690974,-2.160351
50%,2014.000000,145.000000,28.000000,587.500000,17.000000,71.500000,68.000000,6.000000,0.083000,0.179000,...,-0.340000,-0.015000,-0.005117,0.062385,0.141844,0.120951,-0.299616,0.133874,0.417967,0.045613
75%,2016.000000,155.000000,31.000000,652.000000,25.000000,85.000000,85.000000,14.000000,0.105000,0.219000,...,0.527500,0.957500,0.366121,0.822500,0.817740,0.809049,0.457565,0.772735,2.528986,2.423202
max,2017.000000,162.000000,40.000000,754.000000,59.000000,137.000000,139.000000,68.000000,0.206000,0.372000,...,7.140000,6.980000,1.888168,3.665980,3.934791,3.489644,6.189380,3.507457,11.463261,12.516023


In [8]:
data.columns

Index(['Season', 'Name', 'Team', 'G', 'Age', 'PA', 'HR', 'R', 'RBI', 'SB',
       'BB%', 'K%', 'ISO', 'BABIP', 'AVG', 'OBP', 'SLG', 'wOBA', 'wRC+', 'EV',
       'BsR', 'Off', 'O-Swing%', 'Z-Swing%', 'O-Contact%', 'Z-Contact%',
       'GB/FB', 'LD%', 'GB%', 'HR/FB', 'playerid', 'f_val_HR', 'f_val_R',
       'f_val_RBI', 'f_val_SB', 'f_val_BA', 'f_val', 'ny_f_val', 'HR_rate',
       'R_rate', 'RBI_rate', 'SB_rate', 'f_val_HR_rate', 'f_val_R_rate',
       'f_val_RBI_rate', 'f_val_SB_rate', 'f_val_rate', 'ny_f_val_rate',
       'HR_rate_std', 'R_rate_std', 'RBI_rate_std', 'SB_rate_std', 'AVG_std',
       'f_val_std', 'ny_f_val_std'],
      dtype='object')

## Create data structures for use in models

In [9]:
# define columns that won't be used in predictive modeling
drop_cols = ['Name', 'Team', 'f_val_HR', 'f_val_R','f_val_RBI','f_val_SB','f_val_BA', 
             'f_val', 'ny_f_val', 'HR', 'R', 'RBI', 'SB', 'f_val_HR_rate', 'f_val_R_rate', 
             'f_val_RBI_rate', 'f_val_SB_rate', 'f_val_rate', 'ny_f_val_rate', 'HR_rate_std', 
             'R_rate_std', 'RBI_rate_std','SB_rate_std', 'AVG_std', 'f_val_std']

# create DataFrame for use in data visualization
data2 = data.drop(drop_cols, axis=1)

In [10]:
# drop column EV due to missing entries (shows in in-depth data exploration notebook)
data2 = data2.drop('EV', axis=1)

In [11]:
# dropping columns using methodology from in-depth data exploration notebook
drop_cols2 = ['G', 'Age', 'BB%', 'K%', 'BABIP', 'BsR', 'O-Swing%', 
           'Z-Swing%', 'O-Contact%', 'Z-Contact%', 'GB/FB', 'LD%', 
           'GB%', 'SB_rate', 'ISO', 'OBP', 'wOBA', 'wRC+', 'SLG', 'HR/FB']

data3 = data2.drop(drop_cols2, axis=1)

In [14]:
# test on 2 most recent years of data (index 300 is cut off)
# used for all models except Elastic Net
df_train = data3[300:]
df_test = data3[:300]

# use to retain more features for Elastic Net
df_train_enet = data2[300:]
df_test_enet = data2[:300]

In [15]:
# remove target from X
X_train = df_train.drop('ny_f_val_std', axis=1)
X_test = df_test.drop('ny_f_val_std', axis=1)

# use for Elastic Net
X_train_enet = df_train_enet.drop('ny_f_val_std', axis=1)
X_test_enet = df_test_enet.drop('ny_f_val_std', axis=1)

# create target y
y_train = df_train['ny_f_val_std']
y_test = df_test['ny_f_val_std']

In [16]:
# hold onto season column
train_season = X_train['Season']
test_season = X_test['Season']

#hold onto playerid column
train_ID = X_train['playerid']
test_ID = X_test['playerid']

#Now drop the  'playerid' and 'Season' columns since they are unnecessary for the prediction process
X_train.drop(["playerid", "Season"], axis=1, inplace = True)
X_test.drop(["playerid", "Season"], axis=1, inplace = True)
X_train_enet.drop(["playerid", "Season"], axis=1, inplace = True)
X_test_enet.drop(["playerid", "Season"], axis=1, inplace = True)

In [17]:
X_train.head(1)

,PA,AVG,Off,HR_rate,R_rate,RBI_rate
300,654,0.291,23.1,0.124138,0.634483,0.531034


In [18]:
y_train.head()

300    8.210223
301    7.567880
302    6.856675
303    6.657771
304    6.631371
Name: ny_f_val_std, dtype: float64

### Scale the data

In [24]:
# retain column names before changing data structure
X_train_cols = X_train.columns
X_train_enet_cols = X_train_enet.columns

# Create normalized inputs for LinearRegression, XGBoost, and LGBoost
norm_scaler = MinMaxScaler()
X_train_normal = norm_scaler.fit_transform(X_train)
X_test_normal = norm_scaler.transform(X_test)

# create standardized inputs for ENet
stand_scaler = StandardScaler()
X_train_standard = stand_scaler.fit_transform(X_train_enet)
X_test_standard = stand_scaler.transform(X_test_enet)

C:\Users\nkraj\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\nkraj\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\nkraj\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\nkraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  del sys.path[0]
